In [1]:
import os
import time

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn

from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import CosineAnnealingLR

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import matplotlib.pyplot as plt
import seaborn as sns


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [2]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


TRAIN_AUG_CSV_PATH = '/root/data/train_kr_aug.csv'
TRAIN_AUG_IMAGE_PATH = '/root/data/train_aug_patch'

VALID_CSV_PATH = '/root/data/valid.csv'
VALID_IMAGE_PATH = '/root/data/valid'

TEST_CSV_PATH = '/root/data/sample_submission.csv'
TEST_IMAGE_PATH = '/root/data/test'

RESULT_CSV_PATH = '/root/'


# HyperParameter

In [3]:
# training config
img_size = 380
LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 32
num_workers = 0

patience = 5
min_delta = 0.001 # 성능 개선의 최소 변화량

# 1. DATA LOAD

In [4]:
# test image 변환
data_transform = A.Compose([
    A.Resize(height = img_size, width = img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
    ToTensorV2()
])

class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        if self.transform:
            img = self.transform(image=img)['image']
    
        return img, target, name  # 파일 이름 반환

    def get_labels(self):
        return self.df[:, 1]

trn_dataset = ImageDataset(
    TRAIN_AUG_CSV_PATH,
    TRAIN_AUG_IMAGE_PATH,
    transform = data_transform
)

val_dataset = ImageDataset(
    VALID_CSV_PATH,
    VALID_IMAGE_PATH,
    transform = data_transform
)

tst_dataset = ImageDataset(
    TEST_CSV_PATH,
    TEST_IMAGE_PATH,
    transform = data_transform
)

labels = trn_dataset.get_labels()
labels = labels.astype(int)

# DataLoader
trn_loader = DataLoader(
    trn_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = num_workers,
    pin_memory = True,
    drop_last = False
)

val_loader = DataLoader(
    val_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 0,
    pin_memory = True,
    drop_last = False
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = 0,
    pin_memory = True
)

print(len(trn_dataset), len(tst_dataset))

113040 3140


# 2. Model Train

In [5]:
import os
import torch
import numpy as np
import cv2
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision import transforms

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = []
        self._register_hooks()

    def _register_hooks(self):
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0]

        def forward_hook(module, input, output):
            self.activations = output

        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))
        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))

    def __call__(self, x, class_idx):
        self.model.zero_grad()
        output = self.model(x)
        loss = output[:, class_idx].sum()
        loss.backward(retain_graph=True)
        gradients = self.gradients[0].cpu().data.numpy()
        activations = self.activations[0].cpu().data.numpy()
        weights = np.mean(gradients, axis=(1, 2))
        cam = np.zeros(activations.shape[1:], dtype=np.float32)
        for i, w in enumerate(weights):
            cam += w * activations[i, :, :]
        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, x.shape[2:][::-1])
        cam -= np.min(cam)
        cam /= np.max(cam)
        return cam

    def close(self):
        for handle in self.hook_handles:
            handle.remove()

In [6]:
# 모델 생성 및 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('efficientnet_b0',
                        pretrained=True,
                        num_classes = 17).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = LR)


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)
INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


In [7]:
# 한 epoch 학습 함수
def train_one_epoch(loader, model, optimizer, loss_fn, device, epoch):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for step, (image, targets, _) in enumerate(pbar):
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "model": model,
        "train_epoch": epoch,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1
    }

    return ret


In [8]:
# 한 epoch 검증 함수
def valid_one_epoch(loader, model, loss_fn, device, epoch):
    model.eval()
    valid_loss = 0

    preds_list = []
    targets_list = []

    with torch.no_grad():
        pbar = tqdm(loader)
        for step, (image, targets, _) in enumerate(pbar):
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)
       
            valid_loss += loss.item()
        
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

            pbar.set_description(f"Loss: {loss.item():.4f}")

    valid_loss /= len(loader)
    valid_acc = accuracy_score(targets_list, preds_list)
    valid_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "epoch": epoch,
        "valid_loss": valid_loss,
        "valid_acc": valid_acc,
        "valid_f1": valid_f1
    }

    return ret


In [9]:
# 학습 및 조기 종료 설정
f1_scores = []
valid_losses = []
best_valid_f1 = 0  # f1-score는 높을수록 좋으므로 초기값을 0으로 설정
best_model_path = '/root/best_model.pth'

for epoch in range(EPOCHS):
    print(f"{epoch} epoch")
    trn_ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, epoch)
    val_ret = valid_one_epoch(val_loader, model, loss_fn, device, epoch)

    f1_scores.append(val_ret['valid_f1'])
    valid_losses.append(val_ret['valid_loss'])

    print(f"valid loss : {val_ret['valid_loss']}")
    print(f"valid f1 : {val_ret['valid_f1']}")

    # 최적 모델 저장
    if val_ret['valid_f1'] > best_valid_f1 + min_delta:
        best_valid_f1 = val_ret['valid_f1']
        torch.save(model.state_dict(), best_model_path)
        print(f"최적 모델이 {best_model_path}에 저장되었습니다.")
        patience_counter = 0  # 성능이 개선되면 카운터 초기화
    else:
        patience_counter += 1

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"성능 개선이 {patience_counter} 에포크 동안 없었으므로 조기 종료합니다.")
        break

# 최적 모델 로드
best_model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=17).to(device)
best_model.load_state_dict(torch.load(best_model_path, map_location=device))
best_model.eval()
print(f"모델이 {best_model_path}에서 로드되었습니다.")



0 epoch


  0%|          | 0/3533 [00:00<?, ?it/s]

Loss: 1.4773: 100%|██████████| 10/10 [00:01<00:00,  8.91it/s]


valid loss : 0.9269012033939361
valid f1 : 0.8359721163676503
최적 모델이 /root/best_model.pth에 저장되었습니다.
1 epoch


Loss: 2.1984: 100%|██████████| 10/10 [00:01<00:00,  8.99it/s]


...

Loss: 0.8199: 100%|██████████| 10/10 [00:01<00:00,  8.14it/s]


valid loss : 0.6377616211771965
valid f1 : 0.8968100951758475
12 epoch


Loss: 1.0002: 100%|██████████| 10/10 [00:01<00:00,  8.65it/s]


valid loss : 0.9267182528972626
valid f1 : 0.875667202998654
성능 개선이 5 에포크 동안 없었으므로 조기 종료합니다.
모델이 /root/best_model.pth에서 로드되었습니다.


/tmp/ipykernel_54854/1997531126.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path, map_location=device))


In [10]:
# 테스트 셋 예측 및 신뢰도 높은 데이터 선별
preds_list = []
confidences_list = []
image_filenames = []

for images, _, filenames in tqdm(tst_loader):  # 파일 이름을 포함하여 3개 값 반환
    images = images.to(device)
    with torch.no_grad():
        preds = best_model(images)
        confidences = torch.softmax(preds, dim=1).max(dim=1)[0]

    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    confidences_list.extend(confidences.detach().cpu().numpy())
    image_filenames.extend(filenames)

# 임계값 설정 (예: 0.9 이상)
threshold = 0.9
high_confidence_indices = [i for i, confidence in enumerate(confidences_list) if confidence > threshold]
high_confidence_preds = [preds_list[i] for i in high_confidence_indices]
high_confidence_filenames = [image_filenames[i] for i in high_confidence_indices]

print(high_confidence_filenames)
# 신뢰도 높은 데이터 개수 출력
print(f"신뢰도 {threshold} 이상인 데이터 개수: {len(high_confidence_indices)}")


100%|██████████| 99/99 [00:12<00:00,  7.92it/s]

['0008fdb22ddce0ce.jpg', '00091bffdffd83de.jpg', '00396fbc1f6cc21d.jpg', '00901f504008d884.jpg', '009b22decbc7220c.jpg', '00b33e0ee6d59427.jpg', '00bbdcfbbdb3e131.jpg', '00c03047e0fbef40.jpg', '00dcea90f63ad630.jpg', '00e15da96484eb94.jpg', '00f5784903a39fdd.jpg', '0111a6728e9f8a73.jpg', '0114a887a2c2e4ca.jpg', '01385f22f2490868.jpg', '0145dd3d1cd090ae.jpg', '016240faa186d24d.jpg', '016b0c00cdf93e0a.jpg', '017e5da799e1637c.jpg', '0182bffa56bdd844.jpg', '019ed42bb4c2caa9.jpg', '01bd84a54be54b8b.jpg', '01c918594307c6f2.jpg', '01ebd05a14e10618.jpg', '021e7c8d9dc19021.jpg', '0298c2151b43d86b.jpg', '02ac23941313841b.jpg', '02b370d53ff25d45.jpg', '02b3712bd48b8644.jpg', '02b5e73920c3c54e.jpg', '02f79963274b3c41.jpg', '02ffc27eff468793.jpg', '0366fcb21245a5cc.jpg', '039ff76910d52749.jpg', '03a7dcbe6b74bb8e.jpg', '03cb3f2e0962474e.jpg', '040ba9bc68f4e380.jpg', '0412f6a5ba912add.jpg', '04284576791e9ec1.jpg', '0447bde0b7da3a6c.jpg', '046c2f2d42bbac35.jpg', '048efa3d18a4999d.jpg', '048fe083d6a625

In [11]:
# DataFrame 생성
additional_df = pd.DataFrame({
    'ID': high_confidence_filenames,
    'target': high_confidence_preds
})

# 새로운 CSV 파일로 저장
additional_df.to_csv('/root/data/high_confidence_data_patch.csv', index=False)
print(f"신뢰도 0.9 이상인 데이터가 high_confidence_data_patch.csv에 저장되었습니다.")

신뢰도 0.9 이상인 데이터가 high_confidence_data_patch.csv에 저장되었습니다.


In [12]:
# 기존 train.csv 파일 로드
train_df = pd.read_csv('/root/data/train.csv')

# high_confidence_data.csv 파일 로드
high_confidence_df = pd.read_csv('/root/data/high_confidence_data_patch.csv')

# 두 데이터프레임 병합
combined_df = pd.concat([train_df, high_confidence_df], ignore_index=True)

# 새로운 CSV 파일로 저장
combined_csv_path = '/root/data/combined_train_data_patch.csv'
combined_df.to_csv(combined_csv_path, index=False)
print(f"기존 train.csv와 high_confidence_data.csv가 병합된 데이터가 {combined_csv_path}에 저장되었습니다.")

기존 train.csv와 high_confidence_data.csv가 병합된 데이터가 /root/data/combined_train_data_patch.csv에 저장되었습니다.


In [13]:
# 테스트 셋 예측 및 신뢰도 높은 데이터 선별
preds_list = []
confidences_list = []
image_filenames = []

for images, _, filenames in tqdm(tst_loader):  # 파일 이름을 포함하여 3개 값 반환
    images = images.to(device)
    with torch.no_grad():
        preds = best_model(images)
        confidences = torch.softmax(preds, dim=1).max(dim=1)[0]

    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    confidences_list.extend(confidences.detach().cpu().numpy())
    image_filenames.extend(filenames)

# 임계값 설정 (예: 0.9 이상)
threshold = 0.9
high_confidence_indices = [i for i, confidence in enumerate(confidences_list) if confidence > threshold]
high_confidence_preds = [preds_list[i] for i in high_confidence_indices]
high_confidence_filenames = [image_filenames[i] for i in high_confidence_indices]

# 신뢰도 높은 데이터 중 20개만 선택
high_confidence_samples = list(zip(high_confidence_preds, high_confidence_filenames))[:20]

print(high_confidence_filenames)
print(f"신뢰도 {threshold} 이상인 데이터 개수: {len(high_confidence_indices)}")

# Validation 셋 예측 및 잘못된 예측 선별
val_trues_list = []
val_preds_list = []
val_image_filenames = []

for images, labels, filenames in tqdm(val_loader):  # 파일 이름을 포함하여 3개 값 반환
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        preds = best_model(images)

    val_trues_list.extend(labels.detach().cpu().numpy())
    val_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    val_image_filenames.extend(filenames)

# 잘못된 예측 선별
incorrect_samples = [(true, pred, img) for true, pred, img in zip(val_trues_list, val_preds_list, val_image_filenames) if true != pred]



100%|██████████| 99/99 [00:12<00:00,  7.93it/s]


['0008fdb22ddce0ce.jpg', '00091bffdffd83de.jpg', '00396fbc1f6cc21d.jpg', '00901f504008d884.jpg', '009b22decbc7220c.jpg', '00b33e0ee6d59427.jpg', '00bbdcfbbdb3e131.jpg', '00c03047e0fbef40.jpg', '00dcea90f63ad630.jpg', '00e15da96484eb94.jpg', '00f5784903a39fdd.jpg', '0111a6728e9f8a73.jpg', '0114a887a2c2e4ca.jpg', '01385f22f2490868.jpg', '0145dd3d1cd090ae.jpg', '016240faa186d24d.jpg', '016b0c00cdf93e0a.jpg', '017e5da799e1637c.jpg', '0182bffa56bdd844.jpg', '019ed42bb4c2caa9.jpg', '01bd84a54be54b8b.jpg', '01c918594307c6f2.jpg', '01ebd05a14e10618.jpg', '021e7c8d9dc19021.jpg', '0298c2151b43d86b.jpg', '02ac23941313841b.jpg', '02b370d53ff25d45.jpg', '02b3712bd48b8644.jpg', '02b5e73920c3c54e.jpg', '02f79963274b3c41.jpg', '02ffc27eff468793.jpg', '0366fcb21245a5cc.jpg', '039ff76910d52749.jpg', '03a7dcbe6b74bb8e.jpg', '03cb3f2e0962474e.jpg', '040ba9bc68f4e380.jpg', '0412f6a5ba912add.jpg', '04284576791e9ec1.jpg', '0447bde0b7da3a6c.jpg', '046c2f2d42bbac35.jpg', '048efa3d18a4999d.jpg', '048fe083d6a625

100%|██████████| 10/10 [00:01<00:00,  8.17it/s]


In [14]:
# Grad-CAM 시각화를 위한 모델 준비
target_layer = best_model.conv_head
grad_cam = GradCAM(best_model, target_layer)

# Transform 정의
transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])
# 신뢰도 높은 예측에 대해 Grad-CAM 시각화
os.makedirs("/root/over0.9_images_CAM_patch/", exist_ok=True)
for pred, img_path in high_confidence_samples:
    img_name = os.path.basename(img_path)
    src_path = os.path.join("/root/data/test/", img_name)  # test 디렉토리에서 이미지 경로
    img = Image.open(src_path).convert('RGB')
    img = img.resize((img_size, img_size))
    img_tensor = transform(image=np.array(img))['image'].unsqueeze(0).to(device)

    # Grad-CAM 시각화
    cam = grad_cam(img_tensor, class_idx=pred)

    # 원본 이미지와 Grad-CAM 결과 시각화
    image_np = img_tensor.cpu().data.numpy()[0].transpose(1, 2, 0)
    image_np = np.array([0.229, 0.224, 0.225]) * image_np + np.array([0.485, 0.456, 0.406])
    image_np = np.clip(image_np, 0, 1)

    # CAM을 원본 이미지 크기에 맞게 변환
    cam = np.uint8(255 * cam)
    cam = np.uint8(Image.fromarray(cam).resize((image_np.shape[1], image_np.shape[0]), Image.LANCZOS))

    # OpenCV로 색상 맵 적용
    heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)

    # BGR을 RGB로 변환
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # 원본 이미지 위에 히트맵 겹치기
    superimposed_img = heatmap * 0.4 + np.uint8(image_np * 255)

    # 이미지를 0-1 범위로 정규화
    superimposed_img = np.clip(superimposed_img / 255.0, 0, 1)

    # 시각화 이미지 저장
    cam_path = os.path.join("/root/over0.9_images_CAM_patch/", f"cam_{pred}_{img_name}")
    plt.imsave(cam_path, superimposed_img)



/opt/conda/envs/cv/lib/python3.10/site-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


In [15]:
# 잘못된 예측에 대해 Grad-CAM 시각화
os.makedirs("/root/incorrect_images_CAM_patch/", exist_ok=True)
for target, pred, img_path in incorrect_samples:
    img_name = os.path.basename(img_path)
    src_path = os.path.join("/root/data/valid/", img_name)  # test 디렉토리에서 이미지 경로
    img = Image.open(src_path).convert('RGB')
    img = img.resize((img_size, img_size))
    img_tensor = transform(image=np.array(img))['image'].unsqueeze(0).to(device)

    # Grad-CAM 시각화
    cam = grad_cam(img_tensor, class_idx=pred)

    # 원본 이미지와 Grad-CAM 결과 시각화
    image_np = img_tensor.cpu().data.numpy()[0].transpose(1, 2, 0)
    image_np = np.array([0.229, 0.224, 0.225]) * image_np + np.array([0.485, 0.456, 0.406])
    image_np = np.clip(image_np, 0, 1)

    # CAM을 원본 이미지 크기에 맞게 변환
    cam = np.uint8(255 * cam)
    cam = np.uint8(Image.fromarray(cam).resize((image_np.shape[1], image_np.shape[0]), Image.LANCZOS))

    # OpenCV로 색상 맵 적용
    heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)

    # BGR을 RGB로 변환
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # 원본 이미지 위에 히트맵 겹치기
    superimposed_img = heatmap * 0.4 + np.uint8(image_np * 255)

    # 이미지를 0-1 범위로 정규화
    superimposed_img = np.clip(superimposed_img / 255.0, 0, 1)

    # 시각화 이미지 저장
    cam_path = os.path.join("/root/incorrect_images_CAM_patch/", f"cam_{target}_{pred}_{img_name}")
    plt.imsave(cam_path, superimposed_img)

grad_cam.close()

In [16]:
# 예측 결과를 DataFrame으로 변환
test_df = pd.read_csv("/root/data/sample_submission.csv")
pred_df = pd.DataFrame({
    'ID': image_filenames,
    'target': preds_list,
    'confidence': confidences_list
})

In [17]:
# 예측 결과 CSV 파일로 저장
pred_df.to_csv("/root/efficient_net_test_predictions_patch.csv", index=False)

print("예측 결과가 'efficient_net_test_predictions_patch.csv' 파일에 저장되었습니다.")

예측 결과가 'efficient_net_test_predictions_patch.csv' 파일에 저장되었습니다.


# TEST

In [18]:
preds_list = []

best_model.eval()

for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = best_model(image)
        
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv(f"{RESULT_CSV_PATH}/base.csv", index=False)

  0%|          | 0/99 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [ ]:
# 기존 학습 데이터셋
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# 기존 학습 데이터 로더에서 데이터셋 추출
train_images = [item[0] for item in trn_loader.dataset]
train_labels = [item[1] for item in trn_loader.dataset]




NameError: name 'Dataset' is not defined

In [ ]:
# 새로운 학습 데이터 로더 생성
new_trn_loader = DataLoader(new_train_dataset, batch_size=trn_loader.batch_size, shuffle=True)

print(f"새로운 학습 데이터셋 크기: {len(new_train_dataset)}")

# 합친 데이터셋을 새로운 파일로 저장
combined_dataset_path = '/root/combined_dataset.csv'

# 기존 train 데이터 로드
train_df = pd.read_csv('train.csv')

# 추가할 데이터프레임 생성
additional_df = pd.DataFrame({
    'ID': high_confidence_filenames,
    'target': high_confidence_preds
})

# 기존 train 데이터와 추가 데이터 병합
combined_df = pd.concat([train_df, additional_df], ignore_index=True)

# 새로운 CSV 파일로 저장
combined_df.to_csv(combined_dataset_path, index=False)
print(f"합친 데이터셋이 {combined_dataset_path}에 저장되었습니다.")


In [ ]:
# 추가된 데이터로 모델 다시 학습
for epoch in range(EPOCHS):
    print(f"{epoch} epoch (with additional data)")
    trn_ret = train_one_epoch(new_trn_loader, model, optimizer, loss_fn, device, epoch)
    val_ret = valid_one_epoch(val_loader, model, loss_fn, device, epoch)

    f1_scores.append(val_ret['valid_f1'])
    valid_losses.append(val_ret['valid_loss'])
    trained_models.append(trn_ret['model'])

    print(f"valid loss : {val_ret['valid_loss']}")
    print(f"valid f1 : {val_ret['valid_f1']}")

    # 성능 개선 됨
    if val_ret['valid_loss'] < best_valid_loss - min_delta:
        best_valid_loss = val_ret['valid_loss']
        patience_counter = 0  
    # 성능 개선 되지 않음
    else:
        patience_counter += 1  

    # 성능 개선이 patience 만큼 안되면 학습 중단
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

best_model_idx = np.argmin(np.array(valid_losses))
best_model = trained_models[best_model_idx]

In [ ]:
# 최종 예측 결과 저장
preds_list = []

best_model.eval()

for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = best_model(image)
        
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv(f"{RESULT_CSV_PATH}/base.csv", index=False)


# 결과 분석

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix

# 예측 결과 생성
valid_preds_list = []

best_model.eval()

for image, _ in tqdm(val_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = best_model(image)
        
    valid_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

# 실제 레이블과 예측 레이블 준비
true_labels = val_dataset.get_labels()
pred_labels = np.array(valid_preds_list)

# Confusion Matrix 생성
cm = confusion_matrix(true_labels, pred_labels)

# 클래스의 최대값 확인 (히트맵의 크기를 결정하기 위해)
n_classes = max(cm.shape)

# 히트맵 생성
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(n_classes), 
            yticklabels=range(n_classes))

plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# x축과 y축의 눈금을 1단위로 설정
plt.xticks(np.arange(0, n_classes, 1))
plt.yticks(np.arange(0, n_classes, 1))

plt.tight_layout()
plt.show()


100%|██████████| 10/10 [00:01<00:00,  5.90it/s]


ValueError: Classification metrics can't handle a mix of unknown and multiclass targets